In [47]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o yaml | yq

apiVersion: kafka.strimzi.io/v1beta2
kind: KafkaConnector
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"kafka.strimzi.io/v1beta2","kind":"KafkaConnector","metadata":{"annotations":{},"labels":{"strimzi.io/cluster":"my-connect"},"name":"debezium-connector-postgres","namespace":"kafka-main-cluster"},"spec":{"class":"io.debezium.connector.postgresql.PostgresConnector","config":{"converter.type":"avro","database.dbname":"airflow","database.hostname":"postgres-postgresql.postgres.svc","database.password":"${secrets:kafka-main-cluster/postgres-user-debezium:password}","database.port":5432,"database.user":"${secrets:kafka-main-cluster/postgres-user-debezium:username}","key.converter":"io.confluent.connect.avro.AvroConverter","key.converter.schema.registry.url":"http://main-registry-schema-registry.kafka-main-cluster.svc:8081","plugin.name":"decoderbufs","schema.history.internal.kafka.bootstrap.servers":"main-kafka-bootstrap:9092","schema

# Get connector error message

In [37]:
!kubectl get -n kafka-main-cluster kafkaconnectors debezium-connector-postgres -o jsonpath='{.status.connectorStatus.tasks[0].trace}' 

# Using Kubernetes Configuration Provider to load data from Secrets and Config Maps
   * https://strimzi.io/blog/2021/07/22/using-kubernetes-config-provider-to-load-data-from-secrets-and-config-maps/

In [38]:
!kubectl -n kafka-main-cluster get role connector-configuration-role -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"Role","metadata":{"annotations":{},"name":"connector-configuration-role","namespace":"kafka-main-cluster"},"rules":[{"apiGroups":[""],"resourceNames":["postgres-user-debezium","sqlserver-user-debezium"],"resources":["secrets"],"verbs":["get"]}]}
  creationTimestamp: "2024-02-07T10:54:26Z"
  name: connector-configuration-role
  namespace: kafka-main-cluster
  resourceVersion: "8739"
  uid: 93858122-9f56-4696-a632-ef8099cf283e
rules:
  - apiGroups:
      - ""
    resourceNames:
      - postgres-user-debezium
      - sqlserver-user-debezium
    resources:
      - secrets
    verbs:
      - get


In [39]:
!kubectl -n kafka-main-cluster get rolebinding connector-configuration-role-binding -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"rbac.authorization.k8s.io/v1","kind":"RoleBinding","metadata":{"annotations":{},"name":"connector-configuration-role-binding","namespace":"kafka-main-cluster"},"roleRef":{"apiGroup":"rbac.authorization.k8s.io","kind":"Role","name":"connector-configuration-role"},"subjects":[{"kind":"ServiceAccount","name":"my-connect-connect","namespace":"kafka-main-cluster"}]}
  creationTimestamp: "2024-02-07T10:54:26Z"
  name: connector-configuration-role-binding
  namespace: kafka-main-cluster
  resourceVersion: "8742"
  uid: 42b135a2-d412-434a-af75-0693b0108003
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: connector-configuration-role
subjects:
  - kind: ServiceAccount
    name: my-connect-connect
    namespace: kafka-main-cluster


In [40]:
!kubectl -n kafka-main-cluster get secrets postgres-user-debezium -o yaml | yq

apiVersion: v1
data:
  password: ZGViZXppdW0=
  username: ZGViZXppdW0=
kind: Secret
metadata:
  annotations:
    replicator.v1.mittwald.de/replicated-at: "2024-02-07T10:54:26Z"
    replicator.v1.mittwald.de/replicated-from-version: "8759"
    replicator.v1.mittwald.de/replicated-keys: password,username
  creationTimestamp: "2024-02-07T10:54:26Z"
  name: postgres-user-debezium
  namespace: kafka-main-cluster
  resourceVersion: "8760"
  uid: bf4de92f-9979-4253-ac84-eed623ce3a53
type: Opaque


In [41]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/ | jq

[
  {
    "class": "com.github.jcustenborder.kafka.connect.redis.RedisSinkConnector",
    "type": "sink",
    "version": "0.0.0.0"
  },
  {
    "class": "com.mongodb.kafka.connect.MongoSinkConnector",
    "type": "sink",
    "version": "1.11.1"
  },
  {
    "class": "com.splunk.kafka.connect.SplunkSinkConnector",
    "type": "sink",
    "version": "v2.1.1"
  },
  {
    "class": "io.confluent.connect.cassandra.CassandraSinkConnector",
    "type": "sink",
    "version": "2.0.6"
  },
  {
    "class": "io.confluent.connect.elasticsearch.ElasticsearchSinkConnector",
    "type": "sink",
    "version": "14.0.12"
  },
  {
    "class": "io.confluent.connect.http.HttpSinkConnector",
    "type": "sink",
    "version": "1.7.5"
  },
  {
    "class": "io.confluent.connect.mqtt.MqttSinkConnector",
    "type": "sink",
    "version": "1.7.2"
  },
  {
    "class": "io.confluent.connect.s3.S3SinkConnector",
    "type": "sink",
    "version": "10.5.7"
  },
  {
    "class": "io.confluent.connect.sftp.SftpS

In [42]:
!curl -s -H "Accept:application/json" http://my-connect-connect-api.kafka-main-cluster.svc:8083/connector-plugins/source/config/validate | jq

{
  "error_code": 405,
  "message": "HTTP 405 Method Not Allowed"
}


In [43]:
!kubectl -n kafka get serviceaccount strimzi-cluster-operator -o yaml | yq

apiVersion: v1
kind: ServiceAccount
metadata:
  annotations:
    meta.helm.sh/release-name: strimzi-cluster-operator
    meta.helm.sh/release-namespace: kafka
  creationTimestamp: "2024-02-07T10:53:15Z"
  labels:
    app: strimzi
    app.kubernetes.io/managed-by: Helm
    chart: strimzi-kafka-operator-0.39.0
    component: service-account
    heritage: Helm
    release: strimzi-cluster-operator
  name: strimzi-cluster-operator
  namespace: kafka
  resourceVersion: "8226"
  uid: 07c058df-040a-4760-be1a-aeda5f12bc84


In [44]:
!kubectl -n kafka get roles -o yaml | yq

apiVersion: v1
items: []
kind: List
metadata:
  resourceVersion: ""


In [45]:
!kubectl -n kafka get rolebinding  strimzi-cluster-operator -o yaml | yq

apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  annotations:
    meta.helm.sh/release-name: strimzi-cluster-operator
    meta.helm.sh/release-namespace: kafka
  creationTimestamp: "2024-02-07T10:53:15Z"
  labels:
    app: strimzi
    app.kubernetes.io/managed-by: Helm
    chart: strimzi-kafka-operator-0.39.0
    component: role-binding
    heritage: Helm
    release: strimzi-cluster-operator
  name: strimzi-cluster-operator
  namespace: kafka
  resourceVersion: "8253"
  uid: ce51d217-e975-4c11-adf4-0a46ca442af4
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: strimzi-cluster-operator-namespaced
subjects:
  - kind: ServiceAccount
    name: strimzi-cluster-operator
    namespace: kafka
